<a href="https://colab.research.google.com/github/faddednatasha/Sky-Analytics/blob/main/sql_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=bc2a64772f78edfdbff9b246cc6ea64e5815d7308d4a4c20f99dca7d6d485769
  Stored in directory: /root/.cache/pip/wheels/15/a1/e7/6f92f295b5272ae5c02365e6b8fa19cb93f16a537090a1cf27
Successfully built pandasql


In [ ]:
import pandas as pd
from pandasql import sqldf
import numpy as np
# Load the data into a DataFrame
data = pd.read_csv('cleaned_flight_data.csv')

# Define the SQL query interface
pysqldf = lambda q: sqldf(q, globals())


# **AGGREGATE AVERAGE PRICE BY AIRLINE**

In [ ]:
# Average price by airline
query = """
SELECT airline, AVG(price) as avg_price
FROM data
GROUP BY airline
ORDER BY avg_price DESC
"""
result = pysqldf(query)
print(result)


     airline     avg_price
0    Vistara  30396.536302
1  Air_India  23507.019112
2   SpiceJet   6179.278881
3   GO_FIRST   5652.007595
4     Indigo   5324.216303
5    AirAsia   4091.072742


# **AGGREGATE AVERAGE PRICE BY ROUTE**

In [ ]:
data['route'] = data['source_city'] + ' → ' + data['destination_city']

In [ ]:
query = """
SELECT route, ROUND(AVG(price), 2) AS avg_price
FROM data
GROUP BY route
ORDER BY avg_price DESC
Limit 10;
"""
pysqldf(query)

,route,avg_price
0,Chennai → Bangalore,25081.85
1,Kolkata → Chennai,23660.36
2,Bangalore → Kolkata,23500.06
3,Bangalore → Chennai,23321.85
4,Mumbai → Bangalore,23147.87
5,Bangalore → Mumbai,23128.62
6,Mumbai → Chennai,22781.90
7,Chennai → Mumbai,22765.85
8,Kolkata → Bangalore,22744.81
9,Chennai → Kolkata,22669.93


# **IDENTIFY TOP-RATED AND LOWEST-RATED FLIGHTS**

**TOP RATED**

In [ ]:
# Simulate rating between 1.0 and 5.0 if not present
if 'rating' not in data.columns:
    np.random.seed(0)
    data['rating'] = np.random.uniform(2, 5.0, size=len(data)).round(1)


In [ ]:
high_rated_query = """
SELECT airline, flight, source_city, destination_city, price, rating FROM data
group by flight
ORDER BY rating DESC
LIMIT 10
"""
pysqldf(high_rated_query)

,airline,flight,source_city,destination_city,price,rating
0,Vistara,UK-841,Mumbai,Delhi,19383,5.0
1,Vistara,UK-720,Kolkata,Delhi,12054,5.0
2,SpiceJet,SG-8946,Delhi,Bangalore,7425,5.0
3,SpiceJet,SG-8106,Delhi,Kolkata,5954,5.0
4,SpiceJet,SG-5069,Hyderabad,Delhi,6133,5.0
5,SpiceJet,SG-4009,Bangalore,Delhi,12738,5.0
6,AirAsia,I5-881,Delhi,Mumbai,2409,5.0
7,AirAsia,I5-559,Delhi,Bangalore,7423,5.0
8,GO_FIRST,G8-7548,Bangalore,Delhi,7487,5.0
9,GO_FIRST,G8-661,Hyderabad,Delhi,24299,5.0


**LOW RATED FLIGHT**

In [ ]:
lowest_rated_query = """
SELECT airline, flight, source_city, destination_city, price, rating
FROM data
ORDER BY rating ASC, price DESC
LIMIT 10
"""
lowest_rated = pysqldf(lowest_rated_query)
lowest_rated

,airline,flight,source_city,destination_city,price,rating
0,Vistara,UK-705,Delhi,Chennai,99577,2.0
1,Vistara,UK-970,Mumbai,Chennai,96844,2.0
2,Vistara,UK-943,Delhi,Kolkata,93417,2.0
3,Vistara,UK-838,Chennai,Hyderabad,92752,2.0
4,Vistara,UK-705,Delhi,Chennai,92633,2.0
5,Vistara,UK-838,Chennai,Mumbai,90960,2.0
6,Vistara,UK-772,Kolkata,Delhi,90468,2.0
7,Vistara,UK-706,Kolkata,Hyderabad,89897,2.0
8,Vistara,UK-778,Kolkata,Mumbai,89359,2.0
9,Vistara,UK-950,Mumbai,Kolkata,88813,2.0


**FLIGHTS WITH UNUSUAL PRICE VS RATING TRENDS**

In [ ]:
high_price_low_rating_query = """
SELECT airline, flight, source_city, destination_city, price, rating
FROM data
WHERE price > (SELECT AVG(price) FROM data)
  AND rating < (SELECT AVG(rating) FROM data)
ORDER BY price DESC
"""
high_price_low_rating = pysqldf(high_price_low_rating_query)
high_price_low_rating.head(10)

,airline,flight,source_city,destination_city,price,rating
0,Vistara,UK-772,Kolkata,Delhi,123071,3.3
1,Vistara,UK-772,Kolkata,Hyderabad,114705,2.4
2,Vistara,UK-996,Mumbai,Bangalore,114523,2.2
3,Vistara,UK-707,Delhi,Hyderabad,114507,2.6
4,Vistara,UK-996,Mumbai,Chennai,111964,2.3
5,Vistara,UK-996,Mumbai,Chennai,111964,2.7
6,Vistara,UK-850,Bangalore,Delhi,111883,2.3
7,Vistara,UK-775,Mumbai,Delhi,111437,2.7
8,Vistara,UK-910,Mumbai,Bangalore,111163,2.9
9,Vistara,UK-706,Kolkata,Mumbai,110936,2.7


# **LOW PRICES BUT HIGH RATING**

In [ ]:
low_price_high_rating_query = """
SELECT airline, flight, source_city, destination_city, price, rating
FROM data
WHERE price < (SELECT AVG(price) FROM data)
  AND rating > (SELECT AVG(rating) FROM data)
ORDER BY price DESC
"""
low_price_high_rating = pysqldf(low_price_high_rating_query)
low_price_high_rating.head(10)


,airline,flight,source_city,destination_city,price,rating
0,SpiceJet,SG-242,Kolkata,Mumbai,20874,4.0
1,Vistara,UK-776,Kolkata,Mumbai,20874,3.5
2,GO_FIRST,G8-519,Kolkata,Mumbai,20874,3.9
3,Air_India,AI-471,Delhi,Mumbai,20865,4.8
4,Vistara,UK-995,Delhi,Kolkata,20865,4.4
5,Vistara,UK-772,Kolkata,Delhi,20862,3.7
6,Vistara,UK-958,Mumbai,Chennai,20853,3.8
7,Vistara,UK-960,Mumbai,Chennai,20853,4.3
8,Vistara,UK-880,Hyderabad,Kolkata,20833,4.5
9,Air_India,AI-501,Bangalore,Mumbai,20827,4.9
